

### Image segementaion model train 2022 Nov




## 1. Import packages

In [ ]:
!pip install torchvision --upgrade
!pip install grad-cam
!pip install timm
!pip install imagecodecs
!pip install pytorchtools
!pip install git+https://github.com/qubvel/segmentation_models.pytorch

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision.datasets import VisionDataset
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, Subset

import os 
from os import path

import numpy as np
import pandas as pd
from scipy.io import loadmat

from tqdm import tqdm
from PIL import Image

# read tiff
import zipfile
from tifffile import imread
from torchvision.transforms import ToTensor
import random
import csv
import matplotlib.pyplot as plt
import cv2 as cv

In [ ]:
from unet import UNet
from keyholeDataset import Keyhole, KeyholeNoMask
from loss import DiceBCEWithActivationLoss 
from augmentation import get_training_augmentation, preprocess
from utils import plot_2_sidebyside, plot_3_sidebyside, save_model, save_loss_record
from train import train
from validation import validation
import segmentation_models_pytorch as smp


## 2. Initiate a model

In [ ]:
#model = torch.hub.load('milesial/Pytorch-UNet', 'unet_carvana', pretrained=True, scale=0.5)

model = UNet(n_channels=3, n_classes=1, bilinear=1)

# # resnet50, mobilenet_v2,
# model = smp.Unet(
#     encoder_name=None,        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
#     encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
#     in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
#     classes=1,                      # model output channels (number of classes in your dataset)
# )


# model = smp.DeepLabV3(
#     encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
#     encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
#     in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
#     classes=1,                      # model output channels (number of classes in your dataset)
# )


model_name = "Unet_Finetuned"

torch.cuda.empty_cache()
model.cuda()

In [ ]:
#model.segmentation_head = nn.Sequential(*list(model.segmentation_head.children())[:-1])

In [ ]:
# #load model
checkpoint = torch.load("/content/drive/MyDrive/DL_segmentation_models/Unet_Finetune_epoch_50")
model.load_state_dict(checkpoint['model_state_dict'])


## 3. load data + specify batch_size and epochs

In [ ]:
!mkdir Keyhole

from google.colab import drive
drive.mount('/content/drive')

with zipfile.ZipFile('/content/drive/MyDrive/DL_Absorption_data/training_dataset_202206.zip', 'r') as zip:
  zip.extractall(path='/content/Keyhole')

with zipfile.ZipFile('/content/drive/MyDrive/DL_Absorption_data/testing_dataset_202206.zip', 'r') as zip:
  zip.extractall(path='/content/Keyhole')


cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
num_workers = 4 if cuda else 0
print("Cuda = " + str(cuda)+" with num_workers = "+str(num_workers))


In [ ]:
# need to write config file to make this part elegent
batch_size = 1

train_dataset = KeyholeNoMask('/content/Keyhole/training_datase_202206', 
                      transform=None, 
                      preprocess=None )

test_dataset = KeyholeNoMask('/content/Keyhole/testing_dataset_202206', 
                      transform=None, 
                      preprocess=None )


print(f"Train size: {len(train_dataset)}")

print(f"Test size: {len(test_dataset)}")



train_loader = DataLoader(train_dataset, 
                              batch_size=batch_size, 
                              shuffle=False, 
                              num_workers=num_workers)
test_loader = DataLoader(test_dataset, 
                              batch_size=batch_size, 
                              shuffle=False, 
                              num_workers=num_workers)

## 4. Inference

In [ ]:
pred_train_masks = []
for i, batch in enumerate(train_loader):
      print("i = ", i)
      print(batch.shape)
      x = batch.float().to(device)  #.float().to(device) 
      print(x.shape)
      yp = model(x)
      #print("x shape", x.shape) #torch.Size([1, 3, 572, 572])
      #print("y shape", y.shape) #torch.Size([1, 1, 572, 572])


      for i in range(len(x)):
        x_ = x[i].unsqueeze(0)
        yp_ = (yp[i].detach().cpu().numpy()[0]>=0).astype(int)
        pred_train_masks.append(yp_)
        plot_2_sidebyside(x_.detach().cpu().numpy()[0][0], 
                      yp_.astype(int))


In [ ]:
pred_test_masks = []
for i, batch in enumerate(test_loader):
      print("i = ", i)
      x = batch.float().to(device)
      yp = model(x)

      for i in range(len(x)):
        x_ = x[i].unsqueeze(0)
        yp_ = (yp[i].detach().cpu().numpy()[0]>=0).astype(int)
        # plot_2_sidebyside(x_.detach().cpu().numpy()[0][0], 
        #               yp_.astype(int))
        pred_test_masks.append(yp_)

In [ ]:
path = "/content/drive/MyDrive/DL_Absorption_data/train_masks/"
masks = pred_test_masks
for i in range(1, len(masks) + 1):
  mask = masks[i-1]
  # name=ing
  post = ""
  if i < 10:
    post = "00" + str(i)
  elif i < 100:
    post = "0" + str(i)
  else:
    post = str(i)
  image_name = "frame_" + post + ".tif"
  # save
  cv.imwrite(path + image_name, mask)

In [ ]:
len(masks)